In [1]:
import sys, os, getpass

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

user = getpass.getuser()

if user == "sidsel":
    parquet_path = "/home/sidsel/workspace/sparkdata/parquet"
elif user == "svanhmic":
    parquet_path = "/home/svanhmic/workspace/data/DABAI/sparkdata/parquet"

/home/svanhmic/workspace/DABAI/Workflows


In [3]:
#from sample.DataIO import DataIO
#data = DataIO(sc,feature_path=parquet_path+"/featureDataCvr.parquet",company_path=parquet_path+"/companyCvrData")
#feature_data = data.mergeCompanyFeatureData()

In [4]:
#import data!
from pyspark.sql import functions as F
from shared.create_dummy_data import create_dummy_data 

feature_data = create_dummy_data(100, "x y z", "label", outlier_number=0.2, outlier_factor=10)
feature_data = feature_data.select([(10*F.col(i)).alias(i) for i in ["x","y","z"]])

In [5]:
#Select parameters
from cleaning.CreateParametersCleaning import ParamsCleaning

params = ParamsCleaning()

In [6]:
parameters = params.select_parameters()
parameters

In [7]:
print(params.output_parameters(parameters))
#dicKM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'label':('label'), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'label':("cvrNummer","navn"), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}
#dicBKM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'BisectingKMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'label':('label'), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'BisectingKMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'label':("cvrNummer","navn"), 'standardize': False, 'clusters': 2, 'tolerance': 0.0001}

# NOTE!!! Data is note standardized due to random variable is between 0-1 and outliers are beyond
#dicGMM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'GaussianMixture', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'label':('label'), 'standardize': False, 'clusters': 10, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'GaussianMixture', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'label':("cvrNummer","navn"), 'standardize': True, 'clusters': 10, 'tolerance': 0.0001} #
#{'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'initialstep': 10, 'standardize': True, 'clusters': 3, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'Prediction', 'iterations': 10}

{'tol': 0.001, 'k': 2, 'maxIter': 40, 'algorithm': 'GaussianMixture', 'seed': 0}


In [8]:
from cleaning.ExecuteCleaningWorkflow import ExecuteWorkflow

execution_model = ExecuteWorkflow(dict_params=params.output_parameters(parameters)
                                ,cols_features=['x','y','z']
                                ,cols_labels=['label'])


In [10]:
merged_df = execution_model.execute_pipeline(feature_data)

#merged_df.write.parquet('/home/svanhmic/workspace/data/DABAI/sparkdata/parquet/merged_df_parquet')

In [11]:
execution_model.params_labs_feats

{'featuresCol': 'scaled_features',
 'k': 2,
 'maxIter': 40,
 'prediction': 'prediction',
 'probability': 'probability',
 'seed': 0,
 'tol': 0.001}

In [12]:
from cleaning.ShowCleaning import ShowResults

results = ShowResults(execution_model.params_labs_feats, list_features=['x','y','z'], list_labels=['label'] )

AttributeError: module 'scipy' has no attribute 'stats'

In [13]:
results.compute_shift(merged_df).show()

Py4JJavaError: An error occurred while calling o208.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 15.0 failed 1 times, most recent failure: Lost task 4.0 in stage 15.0 (TID 273, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/share/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/share/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/share/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 215, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/share/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 133, in dump_stream
    for obj in iterator:
  File "/usr/local/share/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 204, in _batched
    for item i

In [18]:
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql import types as T
from shared.ComputeDistances import compute_distance
from pyspark.ml.linalg import Vectors, VectorUDT
import numpy as np

win_spec = (Window
            .orderBy(F.col('distance').desc())
            .partitionBy(F.col(execution_model.params_labs_feats['prediction'])))

#percentage_dist = (F.max(merged_df['distance']).over(win_spec)-merged_df['distance'])/100

#udf_distance = F.udf(lambda center, point: compute_distance(center, point), T.DoubleType())
#udf_sub = F.udf(lambda x,y: x-y,VectorUDT())
#dummy = merged_df['centers']-merged_df['centers']
t_udf = F.udf(lambda x,y: float(np.sqrt(np.sum((x-y)*(x-y)))),T.DoubleType())
type_udf = F.udf(lambda x: type(x))
merged_df.select(t_udf(merged_df['centers'],merged_df['scaled_features'])).show(truncate=False)


+----------------------------------+
|<lambda>(centers, scaled_features)|
+----------------------------------+
|42.52239095537738                 |
|35.67710404456799                 |
|63.65161563623139                 |
|48.23645729388438                 |
|50.77613336085953                 |
|35.46163507014414                 |
|38.57514714765584                 |
|59.391561215259046                |
|46.63058748493991                 |
|44.52442869831325                 |
|47.076565181686455                |
|26.894833853368272                |
|44.73705107508782                 |
|49.15992542354263                 |
|49.08445010259027                 |
|44.97862649244513                 |
|34.707397173925614                |
|61.03065303679117                 |
|59.95367070880568                 |
|48.5007627405917                  |
+----------------------------------+
only showing top 20 rows



In [15]:
from cleaning.ShowCleaning import ShowResults

sh = ShowResults()

In [24]:
from pyspark.sql import Window
from pyspark.sql import functions as F
import pandas as pd
gm = runModel.stages[-1].gaussiansDF
pgm = gm.toPandas()
pdf = pd.concat([pgm,pd.DataFrame({'id':(range(6))},columns=['id'])],axis=1)
sqlCtx.createDataFrame()
len(pdf)
#df = spark.createDataFrame(pdf)

6

In [16]:
#widget_select_cluster = widgets.Dropdown(options=['Cluster-'+str(i+1) for i in range(10)])
#display(widget_select_cluster)

# Waring experimental comming up!!!

In [ ]:
import seaborn as sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ax = sb.jointplot(data=pd_results,x='x',y='y')
plt.show()

In [ ]:
from shared import ComputeDistances
from pyspark.sql import types as T
from pyspark.ml.linalg import VectorUDT, Vectors

udf_subtract = F.udf(lambda x, y: Vectors.dense(ComputeDistances.subtract_vectors(x,y)),VectorUDT())
udf_picewise_dist = F.udf(lambda x,y: Vectors.dense(np.abs(x.array-y.array)),VectorUDT())

df = runTest.withColumn("picewise_distance",udf_picewise_dist(F.col("centers"),F.col("features")))

In [ ]:
df.show(5)

In [ ]:
testDf = (df
          .select('prediction','picewise_distance')
          .rdd
          .reduceByKey(lambda x,y : x+y)
          #.mapValues(lambda x: x.array.tolist())
          .map(lambda x: itertools.chain.from_iterable(([x[0]],x[1])))
          .map(lambda x: [float(i) for i in x])
          .toDF(["prediction", 'picewise_distance_x','picewise_distance_y','picewise_distance_z'])
         )

testDf.take(5)

In [ ]:
pdf = testDf.toPandas()
pdf['total'] = pdf['picewise_distance_x']+pdf['picewise_distance_y']


sb.set_style("white")
sb.set_context({"figure.figsize": (24, 10)})

sb.barplot(x='prediction',y=pdf['total'],data=pdf, color = "blue")
bottom_plot = sb.barplot(x='prediction',y=pdf['picewise_distance_x'],data=pdf, color = "red")

topbar = plt.Rectangle((0,0),1,1,fc="red", edgecolor = 'none')
bottombar = plt.Rectangle((0,0),1,1,fc='blue',  edgecolor = 'none')
l = plt.legend([bottombar, topbar], ['Bottom Bar', 'Top Bar'], loc=1, ncol = 2, prop={'size':16})
l.draw_frame(False)

#Optional code - Make plot look nicer
sb.despine(left=True)


plt.xlabel('Clusters')
plt.ylabel('Diff')
plt.show()

In [ ]:
prediction_pdf = pdf['prediction']
cumulative_pdf = pdf.loc[:, 'picewise_distance_x':'picewise_distance_z'].cumsum(axis=1)

In [ ]:
final_pdf = pd.concat([prediction_pdf,cumulative_pdf],axis=1)

In [ ]:
final_pdf

In [ ]:
def make_components_histogram(aggregated_components,feature_list):
    prediction_pdf = pdf['prediction']
    cumulative_pdf = pdf.loc[:, feature_list].cumsum(axis=1)
    pdf = pd.concat([prediction_pdf,cumulative_pdf],axis=1)
    